<a href="https://colab.research.google.com/github/IrinaBoyarchukova/A_B_test/blob/main/Sem__6_%D0%A0%D0%B0%D1%81%D1%87%D1%91%D1%82_%D0%B4%D0%BB%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D0%90_B_%D1%82%D0%B5%D1%81%D1%82%D0%B0_%2B_%D0%BF%D1%80%D0%BE%D0%B4%D0%B2%D0%B8%D0%BD%D1%83%D1%82%D1%8B%D0%B5_%D1%82%D0%BE%D0%BF%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# импорт необходимых библиотек
from statsmodels.stats.power import tt_ind_solve_power, zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.meta_analysis import effectsize_smd
from typing import Union
import plotly.graph_objects as go
from scipy import stats
from math import asin
import numpy as np



In [2]:
#Расчёт effect_size для пропорций
def calc_propotion_effect_size(conv_1: int, nobs_1: int, conv_2: int, nobs_2: int):
    prob_1, prob_2 = conv_1 / nobs_1, conv_2 / nobs_2
    
    es_formula = 2  * asin(np.sqrt(prob_1)) - 2 * asin(np.sqrt(prob_2))
    es_import = proportion_effectsize(prob_1, prob_2)
    
    return es_formula, es_import

#Расчёт effect_size для непрерывных метрик
def calc_continuous_effect_size(mean_1: Union[float, int],
                                std_1: Union[float, int],
                                mean_2: Union[float, int],
                                std_2: Union[float, int],
                                nobs_1: int = 10_000,
                                nobs_2: int = 10_000):
    
    es_formula = (mean_1 - mean_2) / ((std_1**2 + std_2**2) / 2) ** 0.5
    es_import = effectsize_smd(mean_1, std_1, nobs_1, mean_2, std_2, nobs_2)[0]
    
    return es_formula, es_import 

In [3]:
#Расчёт минимально необходимой выборки * 2(для теста и контроля) для пропорций
def calc_sample_size_proportion(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)
#Расчёт минимально необходимой выборки * 2(для теста и контроля) для непрерывной метрики
def calc_sample_size_continuous(effect_size: float,
                                alpha: float = .05,
                                beta: float = .2,
                                ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

In [18]:
# Сборная таблица с кодами
def calc_proportion_es(prob1: float, prob2: float):
    
    return abs(proportion_effectsize(prob1, prob2))

def calc_proportion_es_alt(conv1: float, conv2:float, prob1:float, prob2: float):
    
    return 2  * asin(np.sqrt(conv1/nobs1)) - 2 * asin(np.sqrt(conv2/nobs2))

def calc_continuous_es(mean_control: Union[float, int],
                       std_control: Union[float, int],
                       mean_test: Union[float, int],
                       std_test: Union[float, int]):
    
    return abs(effectsize_smd(mean_control,
                              std_control,
                              1e4,
                              mean_test,
                              std_test,
                              1e4)[0])

def calc_continuous_es_alt(mean_control: Union[float, int],
                           std_control: Union[float, int],
                           mean_test: Union[float, int],
                           std_test: Union[float, int]):
    
    effect_size = (mean_test - mean_control) / ((std_control**2 + std_test**2) / 2) ** 0.5
    return effect_size
    
def calc_sample_size_continuous(effect_size: float,
                     alpha: float = .05,
                     beta: float = .2,
                     ratio: Union[float, int] = 1):
    
    n = tt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

def calc_sample_size_proportion(effect_size: float,
                     alpha: float = .05,
                     beta: float = .2,
                     ratio: Union[float, int] = 1):
    
    n = zt_ind_solve_power(effect_size=effect_size,
                           alpha=alpha,
                           power=(1 - beta),
                           ratio=ratio,
                  )
    return int(n * 2)

Задача 1 Мы хотим провести АБ-тест баннеров. Нынешняя конверсия в клик на баннер (CTR) равна 1.5%. Мы предполагаем, что с новыми баннерами CTR вырастет до 1.7%. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 95%

In [3]:
conv1, conv2 = 0.015, 0.017

es_prop = calc_proportion_es(conv1, conv2)
#es_prop_alt = calc_proportion_es(conv1, conv2)
es_prop

0.015947131645016377

In [4]:
calc_sample_size_proportion(es_prop)

123452

Задача 2 Мы хотим провести АБ-тест формы подтверждения заказа. Нынешняя конверсия в заказ равна 3%. Мы предполагаем, что конверсия вырастет на 0,1%. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 99% и уровнем мощности 90%

In [5]:
conv1, conv2 = 0.03, 0.031

es_prop = calc_proportion_es(conv1, conv2)
#es_prop_alt = calc_proportion_es(conv1, conv2)
es_prop

0.005815545699511593

In [7]:
calc_sample_size_proportion(es_prop, alpha=.01, beta=.1)

1759803

In [18]:
n_conv1, n_conv2 = 164, 228
nobs1, nobs2 = 15500, 15500

conv1, conv2 = n_conv1/nobs1, n_conv2/nobs2
calc_proportion_es(conv1, conv2), calc_proportion_es_alt(n_conv1, n_conv2, nobs1, nobs2)

(0.03707652191393576, -0.03707652191393576)

Задача 3 Дизайнер предложил добавить рекомендательную систему на этапе формирования корзины. Нынешний средний чек равен 2167 рублей, а стандартная ошибка (SD) равна 69. Мы предполагаем, что в новой версии средний чек вырастет 2180. Определите, сколько юзеров нам нужно отправить на каждую версию с уровнем доверия 95%

In [12]:
mu_control, mu_test = 2167, 2180
std_control, std_test = 69, 69 * 1.15

es_cont = calc_continuous_es(mu_control, std_control, mu_test, std_test)
es_cont_alt = calc_continuous_es_alt(mu_control, std_control, mu_test, std_test)
es_cont, es_cont_alt

(0.17482965875688977, 0.17483621585265752)

In [10]:
calc_sample_size_continuous(es_cont), calc_sample_size_continuous(es_cont_alt)

/usr/local/lib/python3.9/dist-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: invalid value encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/usr/local/lib/python3.9/dist-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: invalid value encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


(1029, 1029)

In [15]:
calc_sample_size_continuous(es_cont, alpha=.01, beta=.1), calc_sample_size_continuous(es_cont_alt)

(1950, 1029)

In [5]:
mu_control, mu_test = 2167, 2200
std_control, std_test = 69, 69 * 1.15

calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)

(-0.44381500947213065, -0.4437983645367202)

In [6]:
effect_size = calc_continuous_effect_size(mean_1=mu_control, std_1=std_control, mean_2=mu_test, std_2=std_test)[0]
calc_sample_size_continuous(effect_size=effect_size, alpha=.05, beta=.2)

/usr/local/lib/python3.9/dist-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: invalid value encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)
/usr/local/lib/python3.9/dist-packages/scipy/stats/_continuous_distns.py:6826: RuntimeWarning: invalid value encountered in _nct_cdf
  return np.clip(_boost._nct_cdf(x, df, nc), 0, 1)


161

Задача 4 Вы решили сравниваем метрику деньги на юзера в двух группах. Размер выборки - 1000 элементов в каждой группе. 
Для проверки нормальности распределения на выборке в 1000 наблюдений применили , критерий Шапиро-Уилка и получили p-value, равный 0.00002, alpha = 5% то какой бы вывод мы могли сделать в данном случае?
В этом случае какой статистический критерий для проверки первоначальной гипотезы тут лучше всего подойдёт и почему ?


In [7]:
size = 10000
normal_distr = stats.norm.rvs(size=size)
uniform_distr = stats.uniform.rvs(size=size)
expon_distr = stats.expon.rvs(size=size)

In [8]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=normal_distr, name='normal_distr',))
fig.add_trace(go.Histogram(x=uniform_distr, name='uniform_distr'))
fig.add_trace(go.Histogram(x=expon_distr, name='expon_distr'))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [9]:
distr_name = list(
    zip(
        (normal_distr, uniform_distr, expon_distr),
        ('normal_distr', 'uniform_distr', 'expon_distr')
    )
)

In [10]:
for distr, name in distr_name:
    print(stats.shapiro(distr), name)

ShapiroResult(statistic=0.9998533725738525, pvalue=0.8002068996429443) normal_distr
ShapiroResult(statistic=0.9560269117355347, pvalue=0.0) uniform_distr
ShapiroResult(statistic=0.8191910982131958, pvalue=0.0) expon_distr


/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:1816: UserWarning:

p-value may not be accurate for N > 5000.



In [11]:
for distr, name in distr_name:
    print(stats.kstest(distr, 'norm'), name)

KstestResult(statistic=0.00842405990781403, pvalue=0.4743761349051423, statistic_location=-0.8278563101770778, statistic_sign=1) normal_distr
KstestResult(statistic=0.5000212947048277, pvalue=0.0, statistic_location=5.3377909246421495e-05, statistic_sign=-1) uniform_distr
KstestResult(statistic=0.5000384615056193, pvalue=0.0, statistic_location=9.640869761956065e-05, statistic_sign=-1) expon_distr


In [12]:
for distr, name in distr_name:
    print(stats.kstest(distr, 'uniform'), name)

KstestResult(statistic=0.49359935827619816, pvalue=0.0, statistic_location=6.417238018158813e-07, statistic_sign=1) normal_distr
KstestResult(statistic=0.010260324000127552, pvalue=0.24148429279263484, statistic_location=0.30966032400012755, statistic_sign=-1) uniform_distr
KstestResult(statistic=0.3689776651789678, pvalue=0.0, statistic_location=0.9999776651789678, statistic_sign=-1) expon_distr


In [13]:
for distr, name in distr_name:
    print(stats.kstest(distr, 'expon'), name)

KstestResult(statistic=0.49359935827640405, pvalue=0.0, statistic_location=6.417238018158813e-07, statistic_sign=1) normal_distr
KstestResult(statistic=0.36802238733571524, pvalue=0.0, statistic_location=0.9996115075121887, statistic_sign=1) uniform_distr
KstestResult(statistic=0.005957242883364466, pvalue=0.8677323834302578, statistic_location=0.5708282381298845, statistic_sign=1) expon_distr


In [22]:
!pip install -U distfit
!pip install -U matplotlib

import distfit
print(distfit.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
1.6.9


In [23]:
# pip install distfit
from distfit import distfit

# Initialize for discrete distribution fitting
dfit = distfit()

# Run distfit to and determine whether we can find the parameters from the data.
result = dfit.fit_transform(uniform_distr)

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 2.78062] [loc=0.504 scale=0.288]
[distfit] >INFO> [expon     ] [0.00 sec] [RSS: 5.8014] [loc=0.000 scale=0.504]
[distfit] >INFO> [pareto    ] [0.01 sec] [RSS: 5.8014] [loc=-33554432.000 scale=33554432.000]
[distfit] >INFO> [dweibull  ] [0.11 sec] [RSS: 2.93869] [loc=0.490 scale=0.275]
[distfit] >INFO> [t         ] [1.04 sec] [RSS: 2.77984] [loc=0.504 scale=0.288]
[distfit] >INFO> [genextreme] [0.19 sec] [RSS: 2.49471] [loc=0.425 scale=0.304]
[distfit] >INFO> [gamma     ] [0.18 sec] [RSS: 2.78108] [loc=-63.955 scale=0.001]
[distfit] >INFO> [lognorm   ] [0.45 sec] [RSS: 2.79606] [loc=-14.998 scale=15.500]
[distfit] >INFO> [beta      ] [0.28 sec] [RSS: 0.0231513] [loc=-0.002 scale=1.001]
[distfit] >INFO> [uniform   ] [0.00 sec] [RSS: 0.0291115] [loc=0.000 scale=1.000]
[distfit] >INFO> [loggamma  ] [0.17 sec] [RSS: 2.76115] [loc=-31.343 scale=5.427]
[distfit] >INFO> Compute confidence intervals [

In [24]:
result['model']

{'name': 'beta',
 'score': 0.023151297120039147,
 'loc': -0.0018417382959951308,
 'scale': 1.0014532458081842,
 'arg': (1.0142927493295772, 0.9924329635200211),
 'params': (1.0142927493295772,
  0.9924329635200211,
  -0.0018417382959951308,
  1.0014532458081842),
 'model': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7f0106548df0>,
 'bootstrap_score': 0,
 'bootstrap_pass': None,
 'color': '#e41a1c',
 'CII_min_alpha': 0.05077559262022333,
 'CII_max_alpha': 0.9513441596473019}

Задача 5 Сравниваем метрику конверсия в покупку. Размер выборки - 10000 элементов в каждой группе . Какой(ие) статистический(ие) критерий(ие) тут лучше всего подойдёт(ут) и почему ?

Решение:
Сравниваем метрику конверсии в покупку, выбираем статистический критерий Хи квадрат и z критерий долей.